#### In Sudoku, a "naked single" is a cell that has only one possible number left to fill in. This happens when all other numbers in that cell's row, column, and 3x3 box are already used. So, if you see a cell with just one candidate number, that's your naked single. It's a straightforward but powerful technique to solve puzzles.

Edit：Ran into issue where naked singles were not enough to solve the sudoku, causing unsolved sudoku, to solve this issue, I will combine the naked singles strategy with the brute force backtracking to solve the unsolved sudokus.
Steps to take: Append unsolved quizzes from the original df['quizzes'] and add them to a heap and use the index as it's key

In [1]:
# Convert to 9x9 matrix
def sudoku_board(quiz_string):
    return [list(map(int, quiz_string[i:i+9])) for i in range(0, len(quiz_string), 9)]

In [2]:
# Checks if the move is valid
def is_valid(board, row, col, num):

    # Checking row and column for duplicate num
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    # Check the 3x3 grid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)

    for i in range(start_row, start_row+3):
        for j in range(start_col, start_col+3):
            if board[i][j] == num:
                return False

    return True

In [3]:
# Compares the computed solved board with the solution provided
def check_answer(board, solution):
    solved_board = ''.join(str(cell) for row in board for cell in row)
    return solved_board == solution

In [21]:
def find_hidden_singles(board):
    for num in range(1, 10):
        for i in range(9):
            # Row
            row_candidates = [(i, c) for c in range(9) if board[i][c] == 0 and is_valid(board, i, c, num)]
            if len(row_candidates) == 1:
                board[row_candidates[0][0]][row_candidates[0][1]] = num

            # Column
            col_candidates = [(r, i) for r in range(9) if board[r][i] == 0 and is_valid(board, r, i, num)]
            if len(col_candidates) == 1:
                board[col_candidates[0][0]][col_candidates[0][1]] = num

            # Block
            start_row, start_col = 3 * (i // 3), 3 * (i % 3)
            block_candidates = [(r, c) for r in range(start_row, start_row + 3) for c in range(start_col, start_col + 3) if board[r][c] == 0 and is_valid(board, r, c, num)]
            if len(block_candidates) == 1:
                board[block_candidates[0][0]][block_candidates[0][1]] = num
    return board


# Recursive function with backtracking
def solve_board(board):
    # First, fill in all the naked singles
    find_hidden_singles(board)
    
    for i in range(9):
        for j in range(9):
            # Finds an empty cell
            if board[i][j] == 0:
                # Try all numbers
                for num in range(1, 10):
                    # If move is valid, place number
                    if is_valid(board, i, j, num):
                        board[i][j] = num

                        # Recursively solve the rest of the board
                        if solve_board(board):
                            return True
                        
                        # Reset if the recursion doesn't lead to an answer
                        board[i][j] = 0

                return False
                
    return True


# Recursive function with backtracking
def brute_solve_board(board):
    for i in range(9):
        for j in range(9):
            # Finds an empty cell
            if board[i][j] == 0:
                # Try all numbers
                for num in range(1, 10):
                    if is_valid(board, i, j, num):
                        board[i][j] = num

                        if brute_solve_board(board):
                            return True

                        board[i][j] = 0

                return False
    return True


In [46]:
import time
import heapq

# Function to compare and solve Sudoku boards and measure average time
def compare_sudoku(df):
    total_quizzes = len(df)
    correct_solved = 0
    incorrect_solved = 0
    not_solved = 0
    total_time = 0

    unsolved = []
    
    for index, row in df.iterrows():
        quiz = row['quizzes']
        solution = row['solutions']
        
        # Save quiz and solution as another variable to avoid changes to original df
        original_quiz = sudoku_board(quiz)
        quiz_copy = sudoku_board(quiz)
        solution_board = sudoku_board(solution)
        
        # Start timing
        start_time = time.time()
        
        # Solve the quiz
        if solve_board(quiz_copy):
            # Stop timing
            end_time = time.time()
            
            # Calculate time taken
            elapsed_time = end_time - start_time
            total_time += elapsed_time

            # Compare solved board with provided solution
            if quiz_copy == solution_board:
                correct_solved += 1
            else:
                incorrect_solved += 1

        else:
            not_solved += 1
            heapq.heappush(unsolved, (index, original_quiz)) # Add unsolved quiz to heap

    # Print the results
    print(f"Total quizzes: {total_quizzes}")
    print(f"Correctly solved: {correct_solved}")
    print(f"Incorrectly solved: {incorrect_solved}")
    print(f"Could not be solved: {not_solved}")
    print(f"Total Time with Hidden Singles: {total_time}")
    print("\n")
    
    # Apply brute force to unsolved puzzles
    while unsolved:
        index, original_quiz = heapq.heappop(unsolved)
        start_time = time.time()
        if brute_solve_board(original_quiz):
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            total_time += elapsed_time
            
            # If solved, check if it matches the provided solution
            if original_quiz == sudoku_board(df.loc[index, 'solutions']):
                correct_solved += 1
                not_solved -= 1
            else:
                incorrect_solved += 1
                not_solved -= 1
        else:
            print(f"Entry {index}: Could not be solved with brute force.")
            print(f"{original_quiz}, {sudoku_board(df.loc[index, 'solutions'])}")

    # Calculate the average time
    if total_quizzes > 0:
        average_time = total_time / total_quizzes
    else:
        average_time = 0
    
    # Print final results including brute force attempts
    print(f"After brute force attempt:")
    print(f"Correctly solved: {correct_solved}")
    print(f"Incorrectly solved: {incorrect_solved}")
    print(f"Could not be solved: {not_solved}")
    print(f"Total time for all quiz: {total_time:.4f} seconds")
    print(f"Average time per quiz: {average_time:.4f} seconds")


In [25]:
import pandas as pd
import numpy as np

path = 'sudoku.csv'

df = pd.read_csv(path)
try:
    df = pd.DataFrame({"quizzes":df["puzzle"],"solutions":df["solution"]})
except:
    pass
df.head()

,quizzes,solutions
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [47]:
compare_sudoku(df.head(2000))

Total quizzes: 2000
Correctly solved: 1297
Incorrectly solved: 0
Could not be solved: 703
Total Time with Hidden Singles: 2.462125062942505


After brute force attempt:
Correctly solved: 2000
Incorrectly solved: 0
Could not be solved: 0
Total time for all quiz: 15.2907 seconds
Average time per quiz: 0.0076 seconds


### Finding Hidden Singles:

Time Complexity: This function iterates over the entire 9x9 board multiple times until no single-candidate cells are found. In the worst-case scenario, it might have to scan the entire board 𝑂(𝑛<sup>2</sup>) times where 𝑛=9.

Total Complexity: 
𝑂(𝑛<sup>4</sup>), as each cell might need checking for possible candidates for each number (9 possibilities).



### Brute Force Backtracking:

Time Complexity: The backtracking algorithm for solving Sudoku has an exponential time complexity because it tries to place each number (1 to 9) in empty cells, making recursive calls for possible solutions.

Total Complexity: In the worst case, the complexity can be approximated to 𝑂(9<sup>𝑛<sup>2</sup></sup>) where 𝑛=9.


## In conclusion this method of solving the sudoku is not ideal as the worst case time complexity 𝑂(9<sup>𝑛<sup>2</sup></sup>) would fall to the brute force attempt